Install keras-preprocessing

In [27]:
!pip install keras-preprocessing

Import Libraries

In [28]:
import os
import urllib.request
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator

Download pre-trained weights from InceptionV3

In [29]:
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(
    inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model =  InceptionV3(input_shape=(
    150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

Freeze Pre-trained Model Layers

In [30]:
for layer in pre_trained_model.layers:
        layer.trainable = False

Summaries the pre-trained model

In [31]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 74, 74, 32)           864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 74, 74, 32)           96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

Select Last Layer for Transfer Learning

In [32]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

Download and Extract New Dataset

In [33]:
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
local_file = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/horse-or-human')
zip_ref.close()

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
local_file = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

Define path from data train and valid

In [34]:
train_dir = 'data/horse-or-human'
validation_dir = 'data/validation-horse-or-human'

ImageDataGenerator for Training

In [35]:
train_datagen = ImageDataGenerator(
  rescale=1/255,
  rotation_range=40,
  horizontal_flip=True,
  shear_range=0.2,
  zoom_range=0.2,
  fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
  train_dir,
  class_mode='binary',
  batch_size=64,
  target_size=(150, 150)
)

Found 1027 images belonging to 2 classes.


ImageDataGenerator for Validation

In [36]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(validation_dir,
  class_mode='binary',
  batch_size=32,
  target_size=(150, 150))

Found 256 images belonging to 2 classes.


Build Transfer Learning Model

In [37]:
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

Callback for Early Stopping

In [38]:
class CallbackForTrainAndVal(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') > 0.93 and logs.get('val_acc') > 0.93):
      print("\n Training is stopped accuracy is more than 93%")
      self.model.stop_training = True

custom_callback = CallbackForTrainAndVal()

Compile Model

In [39]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
            loss='binary_crossentropy',
            metrics=['acc'])

Train model

In [40]:
model.fit(
  train_generator,
  epochs=20,
  validation_data=validation_generator,
  callbacks=[custom_callback]
)

Epoch 1/20
17/17 [==============================] - 83s 5s/step - loss: 1.7282 - acc: 0.8870 - val_loss: 0.0518 - val_acc: 0.9883
Epoch 2/20
17/17 [==============================] - ETA: 0s - loss: 0.0486 - acc: 0.9844
 Training is stopped accuracy is more than 93%
17/17 [==============================] - 76s 4s/step - loss: 0.0486 - acc: 0.9844 - val_loss: 0.0147 - val_acc: 0.9922


In [46]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to an image in your validation dataset
image_path = 'data/validation-horse-or-human/horses/horse1-000.png'

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image

# Make prediction
prediction = model.predict(img_array)

# Convert the binary prediction to a class label (0 or 1)
predicted_label = int(np.round(prediction)[0, 0])

# Display the result
if predicted_label == 0:
    print("Predicted: Horse")
else:
    print("Predicted: Human")


1/1 [==============================] - 2s 2s/step
Predicted: Horse
